# 🎵 Music-to-Video Generation using AI  


# 🎬 عنوان پروژه: Music-to-Video Generator  


### 🏫 درس  
**سیستم‌های نهفته و بی‌درنگ**

### 👨‍🏫 استاد  
**مهندس مهدی سیفی‌پور**

### 👥 اعضای تیم  
- سید مهدی منجم  
- علیرضا میرزایی  
- محمدحسین فرهادیان  

### 📅 تاریخ  
تابستان 1404  

---


## ✨ چکیده  

در این پروژه، سامانه‌ای هوشمند طراحی و پیاده‌سازی شده است که قابلیت **تبدیل فایل صوتی موسیقی به ویدئو** را فراهم می‌سازد.  
فرآیند کلی شامل چهار مرحله‌ی اصلی است:  
1. **پردازش صوت** و استخراج متن (اشعار) **Whisper**  
2. **تولید پرامپت خلاقانه** بر اساس متن استخراج‌شده توسط مدل زبانی **Gemini**  
3. **تولید تصویر اولیه** **Stable Diffusion XL**  
4. **تبدیل تصویر به ویدئو** **Stable Video Diffusion**  

خروجی نهایی سامانه، ویدیویی است که با محتوای موسیقی ورودی هماهنگ بوده و جنبه‌های هنری، احساسی و بصری آن را به‌صورت خلاقانه بازآفرینی می‌کند.  

این سیستم نمونه‌ای از **کاربردهای میان‌رشته‌ای هوش مصنوعی** در ترکیب حوزه‌های پردازش زبان طبیعی، بینایی ماشین و تولید محتوای چندرسانه‌ای به شمار می‌رود.  


In [ ]:
!pip install pydub whisper-openai google-generativeai python-dotenv diffusers transformers accelerate torch torchvision imageio[ffmpeg]
!pip install --upgrade Pillow

In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

## 📌 گام یک: استخراج متن  
در نخستین گام، فایل موسیقی ورودی در قالب **ام پی تری** دریافت و برای پردازش‌های بعدی به فرمت **ویو** تبدیل شد.  
سپس این فایل به مدل **ویسپر** داده شد تا محتوای صوتی آن تحلیل شده و متن (اشعار) موجود در موسیقی استخراج گردد.  
در نهایت با استفاده از **جمینی** پرامپت متناسب با موسیقی تولید و ذخیره می‌شود.

این مرحله، پایه و اساس سایر مراحل پروژه محسوب می‌شود؛ زیرا متن استخراج‌شده مبنای تولید پرامپت خلاقانه و در نهایت تولید تصویر و ویدئو قرار می‌گیرد.  


In [ ]:
import os
import whisper
import google.generativeai as genai
from pydub import AudioSegment
from google.colab import userdata

# ---------------- تنظیمات اولیه ----------------
AUDIO_INPUT_PATH = "song.mp3"
WAV_OUTPUT_PATH = "input.wav"
LYRICS_PATH = "lyrics.txt"
PROMPT_PATH = "video_prompt.txt"
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

# ---------------- تبدیل mp3 به wav ----------------
def convert_mp3_to_wav(input_path, output_path):
    print("Converting from MP3 to WAV...")
    try:
        sound = AudioSegment.from_file(input_path)
        sound.export(output_path, format="wav")
        print("Conversion completed.")
        return True
    except Exception as e:
        print(f"Error during MP3 to WAV conversion: {e}")
        return False

# ---------------- استخراج متن با Whisper ----------------
def extract_lyrics(wav_path):
    print("Extracting text with Whisper...")
    try:
        model = whisper.load_model("base")
        result = model.transcribe(wav_path)
        text = result["text"]
        print("Text extracted.")
        return text.strip()
    except Exception as e:
        print(f"Error during text extraction with Whisper: {e}")
        return None

# ---------------- تولید پرامپت با Gemini ----------------
def generate_prompt_from_lyrics(lyrics, api_key):
    print("Producing prompts with Gemini...")
    if not api_key:
        print("GEMINI_API_KEY is not set.")
        return None

    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel("models/gemini-2.0-flash")

        system_prompt = f"""
You are a creative assistant. Based on the lyrics of the song below, create a cinematic script for a text-to-video model. The script should clearly describe the mood, visual style, colors, environment, and emotions.

Lyrics:

{lyrics}

Write it creatively and concisely, suitable for AI video production tools like Stable Video or ModelScope, in a few lines and short.

"""
        response = model.generate_content(system_prompt)
        print("Prompt generated.")
        return response.text.strip()
    except Exception as e:
        print(f"Error during prompt generation with Gemini: {e}")
        return None

# ---------------- اجرای اصلی ----------------
def main():
    if not os.path.exists(AUDIO_INPUT_PATH):
        print(f"Audio file '{AUDIO_INPUT_PATH}' not found.")
        return

    # مرحله 1: تبدیل فایل صوتی
    if not convert_mp3_to_wav(AUDIO_INPUT_PATH, WAV_OUTPUT_PATH):
        return

    # مرحله 2: استخراج متن
    lyrics = extract_lyrics(WAV_OUTPUT_PATH)
    if not lyrics:
        return

    with open(LYRICS_PATH, "w", encoding="utf-8") as f:
        f.write(lyrics)

    # مرحله 3: تولید پرامپت
    prompt = generate_prompt_from_lyrics(lyrics, GEMINI_API_KEY)
    if not prompt:
        print("\nCould not generate prompt. Exiting.")
        return

    # مرحله 4: ذخیره پرامپت
    with open(PROMPT_PATH, "w", encoding="utf-8") as f:
        f.write(prompt)

    print(f"\n--- Prompt generated by Gemini ---\n{prompt}\n----------------------------------")
    print("\nEverything is ready. Lyrics and prompt saved.")

if __name__ == "__main__":
    main()

## 📌 گام دوم: تولید تصویر اولیه

در این مرحله، متنی که توسط مدل جِمینی بر اساس اشعار موسیقی تولید شده بود،
به عنوان پرامپت ورودی به مدل استیبل دیفیوشن ایکس‌ال داده شد.
این مدل وظیفه داشت تا بر اساس توصیفات متنی، یک تصویر اولیه با کیفیت بالا ایجاد کند.

تصویر به دست آمده در این فرآیند، نقطه شروع برای تولید ویدئو محسوب می‌شود.
در این گام همچنین تنظیماتی مانند اندازه تصویر و کنترل بذر تصادفی انتخاب شد
تا تکرارپذیری و یکپارچگی در نتایج تضمین گردد.

در نتیجه، خروجی این مرحله یک تصویر است که ارتباط مستقیمی با فضای معنایی
اشعار موسیقی دارد و مبنای گام سوم یعنی تولید ویدئو قرار می‌گیرد.

In [ ]:
# @title Default title text
!pip install diffusers transformers accelerate torch torchvision imageio[ffmpeg]
!pip install --upgrade Pillow

In [ ]:
import os
import torch
from diffusers import DiffusionPipeline
from PIL import Image

# ---------------- تنظیمات اولیه ----------------
PROMPT_PATH = "video_prompt.txt"
INITIAL_IMAGE_PATH = "initial_image.png"

# ---------------- تولید تصویر با Stable Diffusion XL ----------------
def generate_initial_image(prompt, output_path):
    print("Generating initial image with Stable Diffusion XL...")
    try:
        # Load the text-to-image model (SDXL)
        txt2img_pipe = DiffusionPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype=torch.float16,
            variant="fp16"
        )
        txt2img_pipe.to("cuda")

        # Generate the initial image from the prompt
        generator = torch.manual_seed(42)
        initial_image = txt2img_pipe(prompt=prompt, generator=generator).images[0]
        initial_image.save(output_path)
        print("Initial image generated and saved.")

        # Clear GPU memory
        del txt2img_pipe
        torch.cuda.empty_cache()

        return True
    except Exception as e:
        print(f"Error during image generation: {e}")
        return False

# ---------------- اجرای اصلی برای تولید تصویر اولیه ----------------
def main_image_generation():
    if not os.path.exists(PROMPT_PATH):
        print(f"Prompt file '{PROMPT_PATH}' not found. Please run the first cell to generate the prompt.")
        return

    with open(PROMPT_PATH, "r", encoding="utf-8") as f:
        prompt = f.read()

    print(f"\n--- Using saved prompt to generate initial image ---\n{prompt}\n----------------------------------")

    if generate_initial_image(prompt, INITIAL_IMAGE_PATH):
        print("\nInitial image generation completed successfully.")
    else:
        print("\nInitial image generation failed.")

if __name__ == "__main__":
    main_image_generation()

##📌 گام سوم: تولید ویدئو

در این مرحله، تصویر اولیه تولیدشده توسط مدل استیبل دیفیوشن ایکس‌ال به عنوان ورودی به مدل استیبل ویدئو دیفیوشن داده شد تا یک ویدئو کوتاه ایجاد گردد.
این مدل بر اساس توصیفات پرامپت و ویژگی‌های تصویر اولیه، فریم‌های متوالی و حرکت‌های طبیعی را تولید می‌کند.

تنظیماتی مانند تعداد فریم‌ها، اندازه و رزولوشن تصویر، و شدت نویز حرکتی اعمال شد تا ویدئو نهایی از نظر بصری هماهنگ و روان باشد.
خروجی این مرحله، یک ویدئو هنری و خلاقانه است که بیانگر فضای معنایی و احساسی اشعار موسیقی بوده و نتیجه نهایی پروژه را تشکیل می‌دهد.

❌**محدودیت این قسمت میزان استفاده از منابع پردازشی گرافیکی ارائه شده توسط گوگل کلب بوده که برای تولید ویدئو تنظیمات خروجی روی کیفیت پایین تنظیم شده است❌**

In [ ]:
import os
import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video
from PIL import Image
import imageio

# ---------------- تنظیمات اولیه ----------------
INITIAL_IMAGE_PATH = "initial_image.png"
VIDEO_OUTPUT_PATH = "generated_video.mp4"

# ---------------- تولید ویدیو با Stable Video Diffusion ----------------
def generate_video_from_image(image_path, output_path):
    print("Generating video from the initial image with Stable Video Diffusion...")
    try:
        # بارگذاری مدل ساخت ویدیو
        pipe = StableVideoDiffusionPipeline.from_pretrained(
            "stabilityai/stable-video-diffusion-img2vid-xt",
            torch_dtype=torch.float16,
            variant="fp16"
        )
        pipe.to("cuda")

        # بارگذاری تصویر اولیه
        initial_image = load_image(image_path)

        # --- تغییر اول: کاهش ابعاد تصویر ---
        # ما رزولوشن رو برای کاهش مصرف حافظه کمتر می‌کنیم
        target_size = 512
        width, height = initial_image.size
        if height > width:
            new_height = target_size
            new_width = int(new_height * width / height)
        else:
            new_width = target_size
            new_height = int(new_width * height / width)

        new_width = new_width - new_width % 8
        new_height = new_height - new_height % 8

        initial_image = initial_image.resize((new_width, new_height))
        print(f"Image resized to: {initial_image.size}")


        # تولید فریم‌های ویدیو
        # --- تغییر دوم: کاهش decode_chunk_size ---
        video_frames = pipe(
            initial_image,
            num_frames=25,
            decode_chunk_size=4, # این عدد از 8 به 4 کاهش یافت
            motion_bucket_id=127,
            noise_aug_strength=0.1
        ).frames[0]

        print("Video frames generated. Saving the video...")

        # ذخیره فریم‌ها به صورت فایل ویدیویی mp4
        export_to_video(video_frames, output_path, fps=7)

        # پاک‌سازی حافظه GPU
        del pipe
        torch.cuda.empty_cache()

        print(f"Video saved successfully at: {output_path}")
        return True

    except Exception as e:
        print(f"Error during video generation: {e}")
        return False

# ---------------- اجرای اصلی برای تولید ویدیو ----------------
def main_video_generation():
    if not os.path.exists(INITIAL_IMAGE_PATH):
        print(f"Initial image '{INITIAL_IMAGE_PATH}' not found. Please run the previous cells first.")
        return

    if generate_video_from_image(INITIAL_IMAGE_PATH, VIDEO_OUTPUT_PATH):
        print("\nVideo generation completed successfully.")
    else:
        print("\nVideo generation failed.")

if __name__ == "__main__":
    main_video_generation()